<a href="https://colab.research.google.com/github/yavuzim/beautifulsoup/blob/main/beautifulsoup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***BeautifulSoup ile Kategori Altındaki Haber Linklerinin Toplanması***

In [63]:
#Kütüphanelerimizi içeriye alalım.
#Veri İşlemleri
import pandas as pd
import numpy as np
#Veri Kazıma
from bs4 import BeautifulSoup as bs
import requests
import datetime
#Konsol Güzelleştirme
from progressbar import *
from IPython.display import clear_output

***Dunyahalleri.com’un 6 farklı ana kategorisi var, öncelikle bu kategorilere gireceğiz. Ardından sayfa sayfa bütün makalelerin linklerini toplayacağız.***

In [64]:
#Kategoriler
sections = ["https://www.dunyahalleri.com/category/haftanin-ozeti/page/",
            "https://www.dunyahalleri.com/category/genel-gundem/page/",
            "https://www.dunyahalleri.com/category/teknoloji-bilim/page/",
            "https://www.dunyahalleri.com/category/internet-girisimler/page/",
            "https://www.dunyahalleri.com/category/tasarim-inovasyon/page/",
            "https://www.dunyahalleri.com/category/kultur-sanat/page/"]

In [65]:
urls = []
#Öncelikle bir Kategori seçiyoruz.
for section in sections:
    #Kategorinin içerisinde sırayla 100 sayfa gezineceğiz.
    for i in range(1,2):
        try:
            #Öncelikle URL'imizi oluşturuyoruz. Örneğin;
            #https://www.dunyahalleri.com/category/kultur-sanat/page/25
            newurl = section+str(i)
            print(newurl)

            #Url'nin içerisindeki bütün html dosyasını indiriyoruz.
            html = requests.get(newurl).text
            soup = bs(html, "lxml")
            #Yukarıdaki şemadada görüldüğü gibi bütün makaleler bu  element'in içerisinde yer alıyor.
            #Bizde bütün makaleleri buradan tags adında bir değişkene topluyoruz.
            tags = soup.findAll("div", class_="wp-site-blocks")[0]

            #Sırayla bütün makalelere girip, href'in içerisindeki linki urls adlı listemize append ediyoruz.
            for a in tags.find_all('a', href=True):
                urls.append((section.split("/")[4],a['href']))
        except IndexError:
            break

https://www.dunyahalleri.com/category/haftanin-ozeti/page/1
https://www.dunyahalleri.com/category/genel-gundem/page/1
https://www.dunyahalleri.com/category/teknoloji-bilim/page/1
https://www.dunyahalleri.com/category/internet-girisimler/page/1
https://www.dunyahalleri.com/category/tasarim-inovasyon/page/1
https://www.dunyahalleri.com/category/kultur-sanat/page/1


In [66]:
urldata = pd.DataFrame(urls)
print(urldata)
urldata.columns = ["Kategori","Link"]
urldata.head()

                  0                                                  1
0    haftanin-ozeti                      https://www.dunyahalleri.com/
1    haftanin-ozeti  https://www.dunyahalleri.com/category/haftanin...
2    haftanin-ozeti  https://www.dunyahalleri.com/tag/haftanin-paza...
3    haftanin-ozeti  https://www.dunyahalleri.com/tag/haftanin-spor...
4    haftanin-ozeti  https://www.dunyahalleri.com/category/genel-gu...
..              ...                                                ...
453    kultur-sanat          https://www.dunyahalleri.com/search_gcse/
454    kultur-sanat              https://www.twitter.com/dunya_halleri
455    kultur-sanat             https://www.instagram.com/dunyahalleri
456    kultur-sanat           https://www.facebook.com/dunyahallerimiz
457    kultur-sanat               https://www.youtube.com/dunyahalleri

[458 rows x 2 columns]


,Kategori,Link
0,haftanin-ozeti,https://www.dunyahalleri.com/
1,haftanin-ozeti,https://www.dunyahalleri.com/category/haftanin...
2,haftanin-ozeti,https://www.dunyahalleri.com/tag/haftanin-paza...
3,haftanin-ozeti,https://www.dunyahalleri.com/tag/haftanin-spor...
4,haftanin-ozeti,https://www.dunyahalleri.com/category/genel-gu...


In [67]:
#Bazı linkler çoklamışlar, onlardan kurtulmak için drop_duplicates() #Fonksiyonunu kullanacağım.
urldata = urldata.drop_duplicates()
urldata.to_csv('urldata.csv')

In [72]:
def GetData(url):
    try:
        print("url : ", url)
        # URL içerisindeki HTML'i indiriyoruz.
        html = requests.get(url).text
        soup = bs(html, "lxml")

        # Belirlediğimiz element'in altındaki bütün <p> etiketlerini seçiyoruz.
        body_text = soup.findAll(
            "div",
            class_="entry-content wp-block-post-content is-layout-flow wp-container-core-post-content-is-layout-1 wp-block-post-content-is-layout-flow"
        )[0].findAll('p')

        # Body_text adındaki metni tek bir string üzerinde topluyoruz.
        body_text_big = ""
        for i in body_text:
            body_text_big += i.text

        # Başlığı ve zamanı seçiyoruz.
        header_tag = soup.find(
            "h1",
            class_=["has-text-align-left", "wp-block-post-title", "has-text-align-left wp-block-post-title"]
        )
        header = header_tag.text if header_tag else "Bilinmeyen Başlık"

        summarized = soup.find("div", class_="taxonomy-category wp-block-post-terms").text
        print(summarized)
        # Zaman etiketi kontrol ediliyor.
        timestamp_tag = soup.find("div", class_="has-text-align-left wp-block-post-date")
        timestamp = timestamp_tag.text if timestamp_tag else "Bilinmeyen Zaman"

        return (url, header, body_text_big,summarized, timestamp)

    except IndexError:
        # Link boş ise verilen hata üzerine Boş Data mesajını dönüyor.
        return ("Boş Data", "", "", "")

    except Exception as e:
        # Beklenmedik hataları yönetmek için genel bir yakalama.
        print(f"Hata: {e}")
        return (url, "Hata", "", "")


In [79]:
from tqdm import tqdm

bigdata = []
k = 0
for i in tqdm(urldata.Link):
    clear_output(wait=True)
    print(k)
    bigdata.append(GetData(i))
    k = k + 1

bigdatax = pd.DataFrame(bigdata)

# DataFrame'in satır indekslerini ve sütun adlarını görüntüleme
print(bigdatax.index)  # bigdatax'in satır indeksleri
print(bigdatax.columns)  # bigdatax'in sütun adları
print(bigdatax.shape[1])  # Sütun sayısını döndürür

# Mevcut sütunları kontrol ederek silme
columns_to_drop = [5, 6, 7]
# existing_columns = [col for col in columns_to_drop if col in bigdatax.columns]
# bigdatax.drop(existing_columns, axis=1, inplace=True)

# "B" değerine sahip satırları kaldırma
bigdatax.drop(bigdatax[bigdatax[0] == "B"].index, axis=0, inplace=True)

# Sütun isimlerini atama
bigdatax.columns = ["Link", "Başlık", "Body_text", "Summarized_Text","TimeStamp"]

# Benzersiz bağlantılara göre filtreleme
bigdatax = bigdatax.drop_duplicates(subset="Link")

# İndeksleri sıfırlama
bigdatax.index = range(0, len(bigdatax))

# İlk birkaç satırı görüntüleme
bigdatax


275
url :  https://www.youtube.com/dunyahalleri


100%|██████████| 276/276 [01:05<00:00,  4.22it/s]

RangeIndex(start=0, stop=276, step=1)
RangeIndex(start=0, stop=5, step=1)
5


,Link,Başlık,Body_text,Summarized_Text,TimeStamp
0,Boş Data,,,,None
1,https://www.dunyahalleri.com/yeniden-merhaba/,Yeniden merhaba!,Küresel trendleri belirleyen gelişmeler hakkın...,Genel Gündem,12/09/2021
2,https://www.dunyahalleri.com/hoscakalin/,Hoşçakalın,Dünya Halleri yayın hayatına 2 Kasım 2014 tari...,Genel Gündem,06/08/2019
3,https://www.dunyahalleri.com/haftanin-spor-oze...,Haftanın Spor Özeti: 59,"Haftanın Spor Özeti, Dünya Halleri özetlerinin...",Genel Gündem,05/08/2019
4,https://www.dunyahalleri.com/haftanin-ozeti-249/,Haftanın Özeti: 249,Dünyada bir yanda açlık çeken insanlar diğer y...,Haftanın Özeti,04/08/2019
...,...,...,...,...,...
64,https://www.dunyahalleri.com/yeni-orumcek-adam...,Yeni Örümcek Adam filmi beklenenden çok daha f...,"Örümcek Adam: Evden Uzakta filmi, 2 Temmuz’da ...",Kültür / Sanat,08/07/2019
65,https://www.dunyahalleri.com/spotify-sanatcila...,Spotify sanatçıların kendi müziklerini yükleme...,"Bağımsız sanatçılar, artık kendi müziklerini d...",Kültür / Sanat,06/07/2019
66,https://www.dunyahalleri.com/dunyanin-ilk-sara...,Dünyanın ilk şarap tema parkı Fransa’da açıldı,"Dünyanın ilk şarap tema parkı, Fransa’nın Bord...",Kültür / Sanat,06/07/2019
67,https://www.dunyahalleri.com/raspberry-pi-ile-...,Raspberry Pi ile çalışan taşınabilir synthesizer,Küçük bir gruba hitap ediyor olsa da Critter &...,Kültür / Sanat,05/07/2019
